In [ ]:
import pandas as pd
import sys
from pathlib import Path
from tqdm.auto import tqdm

sys.path.append(str(Path.cwd().parent.parent))
from utils import DATA_DIR

ds_large = pd.read_parquet(DATA_DIR / "datasets" / "large" / "combined.parquet")

# Split at 399 segments
# sm_train, sm_test = train_test_split(ds_small, test_size=0.4, random_state=SEED)
# pd.DataFrame(sm_train).to_parquet(
#     DATA_DIR / "datasets" / "small" / "train.parquet", index=True
# )
# pd.DataFrame(sm_test).to_parquet(
#     DATA_DIR / "datasets" / "small" / "test.parquet", index=True
# )

/home/terra/Projects/vis-desc/modules/lab/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Split at 500 segments
ds_small = pd.read_parquet(DATA_DIR / "datasets" / "small" / "agreed.parquet")
# sm_train = pd.read_parquet(DATA_DIR / "datasets" / "small" / "train.parquet")
# sm_test = pd.read_parquet(DATA_DIR / "datasets" / "small" / "test.parquet")

# # Exclude old train and test from agreed (key by text)
# ds_small = ds_small[
#     ~ds_small["text"].isin(sm_train["text"])
#     & ~ds_small["text"].isin(sm_test["text"])
# ]
# print(len(ds_small))

# sm_train2, sm_test2 = train_test_split(ds_small, test_size=0.4, random_state=SEED)
# sm_train2 = pd.concat([sm_train, sm_train2])
# sm_test2 = pd.concat([sm_test, sm_test2])

# # Put last row from test2 to train2 to keep 300:200 ratio
# sm_train2 = pd.concat([sm_train2, sm_test2.iloc[[-1]]])
# sm_test2 = sm_test2.iloc[:-1]

# sm_train2.to_parquet(
#     DATA_DIR / "datasets" / "small" / "train2.parquet", index=True
# )
# sm_test2.to_parquet(
#     DATA_DIR / "datasets" / "small" / "test2.parquet", index=True
# )

### ModernBERT

In [3]:
from text2features import FeatureService
from text2features_paths import (
    FEATURE_PIPELINE_RESOURCES,
    FEATURE_SERVICE_MODERNBERT_ONNX_PATH,
)
from sklearn.utils import gen_batches

feature_service = FeatureService(
    feature_pipeline_resources=FEATURE_PIPELINE_RESOURCES,
    modernbert_onnx_path=FEATURE_SERVICE_MODERNBERT_ONNX_PATH,
)

BATCH_SIZE = 32

2025-10-25 23:50:25.402142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761429025.499564  963635 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761429025.528790  963635 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-25 23:50:25.830611: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


using device cpu


Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 58720.26it/s]


In [ ]:
batches = gen_batches(len(ds_small), BATCH_SIZE)
embedding_series = []

for batch in tqdm(batches, desc="Extracting embeddings"):
    texts = ds_small["text"].iloc[batch].tolist()
    features = feature_service.get_modernbert_embeddings(texts)
    embedding_series.extend(features)

embedding_df = ds_small[["text"]].copy()
embedding_df["cls"] = embedding_series
embedding_df.to_parquet(
    DATA_DIR / "datasets" / "small" / "modernbert_cls_embeddings.parquet", index=False
)

Extracting embeddings: 11it [04:53, 22.42s/it]

: 

: 

In [ ]:
batches = gen_batches(len(ds_large), BATCH_SIZE)
embedding_series = []

for batch in tqdm(batches, desc="Extracting embeddings"):
    texts = ds_large["text"].iloc[batch].tolist()
    features = feature_service.get_modernbert_embeddings(texts)
    embedding_series.extend(features)

embedding_df = ds_large[["text"]].copy()
embedding_df["cls"] = embedding_series
embedding_df.to_parquet(
    DATA_DIR / "datasets" / "large" / "modernbert_cls_embeddings.parquet", index=False
)

Extracting embeddings: 1563it [11:18:16, 26.04s/it]


### MiniLM

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

2025-10-25 19:50:31.030267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761414631.082866  788025 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761414631.113630  788025 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-25 19:50:31.212796: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
embedding_df = ds_small[["text"]].copy()
embedding_df["cls"] = model.encode(ds_small["text"].tolist()).tolist()
embedding_df.to_parquet(
    DATA_DIR / "datasets" / "small" / "minilm_embeddings.parquet", index=False
)

In [ ]:
embedding_df = ds_large[["text"]].copy()
embedding_df["cls"] = model.encode(ds_large["text"].tolist()).tolist()
embedding_df.to_parquet(
    DATA_DIR / "datasets" / "large" / "minilm_embeddings.parquet", index=False
)

### Validate saved data

In [1]:
import pandas as pd
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent.parent))
from utils import DATA_DIR

sm_minilm = pd.read_parquet(
    DATA_DIR / "datasets" / "small" / "minilm_embeddings.parquet"
)
lg_minilm = pd.read_parquet(
    DATA_DIR / "datasets" / "large" / "minilm_embeddings.parquet"
)
sm_modernbert = pd.read_parquet(
    DATA_DIR / "datasets" / "small" / "modernbert_cls_embeddings.parquet"
)
lg_modernbert = pd.read_parquet(
    DATA_DIR / "datasets" / "large" / "modernbert_cls_embeddings.parquet"
)

In [ ]:
SM_LENGTH = 500
LG_LENGTH = 100000

assert len(sm_minilm) == SM_LENGTH, f"{len(sm_minilm)} != {SM_LENGTH}"
assert len(lg_minilm) == LG_LENGTH, f"{len(lg_minilm)} != {LG_LENGTH}"
assert len(sm_modernbert) == SM_LENGTH, f"{len(sm_modernbert)} != {SM_LENGTH}"
assert len(lg_modernbert) == LG_LENGTH, f"{len(lg_modernbert)} != {LG_LENGTH}"

# Duplicates in 'text' column
assert sm_minilm["text"].nunique() == SM_LENGTH
assert sm_minilm["cls"].apply(tuple).nunique() == SM_LENGTH
assert lg_minilm["text"].nunique() == LG_LENGTH
assert lg_minilm["cls"].apply(tuple).nunique() == LG_LENGTH
assert sm_modernbert["text"].nunique() == SM_LENGTH
assert sm_modernbert["cls"].apply(tuple).nunique() == SM_LENGTH
assert lg_modernbert["text"].nunique() == LG_LENGTH
assert lg_modernbert["cls"].apply(tuple).nunique() == LG_LENGTH

# NaNs in 'cls' lists
assert not sm_minilm.explode("cls")["cls"].isna().any()
assert not lg_minilm.explode("cls")["cls"].isna().any()
assert not sm_modernbert.explode("cls")["cls"].isna().any()
assert not lg_modernbert.explode("cls")["cls"].isna().any()

NameError: name 'lg_minilm' is not defined